# Analyzing a PAC
I'm trying to understand the spending habits of the Microsoft PAC since I work there. The following is primarily based on the code from [this workbook](https://github.com/boblannon/blogpost_fec-api-howto/blob/master/fec_api.ipynb) from [Bob Lannon](https://github.com/boblannon) that explains how to use the FEC API.


First I exported the [spending data for the Microsoft PAC from 2017-18](https://www.fec.gov/data/committee/C00227546/?tab=spending). This got me a csv file used later. This wasn't really enough to sort out what I was interested in as there is no party affiliation in the exported data. So that is what I wanted to use the FEC API to fill in.

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
import numpy as np
import pandas as pd
import requests
import os
import json
from copy import deepcopy

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s')
logging.getLogger("requests").setLevel(logging.ERROR) # silencing requests logging 

# Logging for this notebook
logger = logging.getLogger()
logger.setLevel(logging.INFO) # set this to whatever you'd like

Globals for the base url of the web api and a file to store your API key.

In [4]:
BASE_URL = 'http://api.open.fec.gov/v1'
API_KEY = open(os.path.expanduser('~/.api-keys/data.gov'),'r').read().strip()

Helper functions from Bob's original notebook, no changes.

In [5]:
def all_results(endpoint, params):
    _params = deepcopy(params)
    _params.update({'api_key': API_KEY})
    _url = BASE_URL+endpoint
    logging.info('querying endpoint: {}'.format(_url))
    
    initial_resp = requests.get(_url, params=_params)
    
    logging.debug('full url eg: {}'.format(initial_resp.url))
            
    initial_data = initial_resp.json()
    
    num_pages = initial_data['pagination']['pages']
    num_records = initial_data['pagination']['count']
    logging.info('{p} pages to be retrieved, with {n} records'.format(
            p=num_pages, n=num_records))
    
    current_page = initial_data['pagination']['page']
    logging.debug('page {} retrieved'.format(current_page))
    
    for record in initial_data['results']:
        yield record
    
    while current_page < num_pages:
        current_page += 1
        _params.update({'page': current_page})
        _data = requests.get(_url, params=_params).json()
        logging.debug('page {} retrieved'.format(current_page))
        for record in _data['results']:
            yield record
            
    logging.info('all pages retrieved')
    
def count_results(endpoint, params):
    _params = deepcopy(params)
    _params.update({'api_key': API_KEY})
    _url = BASE_URL+endpoint
    
    _data = requests.get(_url, params=_params).json()
    
    return _data['pagination']['count']

First let'sfind the Microsoft PAC using the API

In [6]:
q_committee = {
    'q': 'microsoft',
}

[r for r in all_results('/committees/', q_committee)]

2018-07-17 22:33:33,801 : INFO : querying endpoint: http://api.open.fec.gov/v1/committees/
2018-07-17 22:33:34,680 : INFO : 1 pages to be retrieved, with 2 records
2018-07-17 22:33:34,683 : INFO : all pages retrieved


[{'designation_full': 'Unauthorized',
  'last_f1_date': '1995-03-01',
  'committee_type_full': 'PAC - Nonqualified',
  'name': 'COMMITTEE TO FIGHT MICROSOFT CORP;THE',
  'organization_type_full': None,
  'committee_id': 'C00301309',
  'cycles': [1996],
  'party': None,
  'state': 'FL',
  'treasurer_name': 'ANTHONY R MARTIN',
  'organization_type': None,
  'designation': 'U',
  'first_file_date': '1995-03-01',
  'party_full': None,
  'committee_type': 'N',
  'last_file_date': '1996-01-26',
  'filing_frequency': 'T',
  'candidate_ids': []},
 {'designation_full': 'Lobbyist/Registrant PAC',
  'last_f1_date': '2018-02-08',
  'committee_type_full': 'PAC - Qualified',
  'name': 'MICROSOFT CORPORATION POLITICAL ACTION COMMITTEE',
  'organization_type_full': 'Corporation',
  'committee_id': 'C00227546',
  'cycles': [2008,
   2006,
   1998,
   1988,
   1996,
   1992,
   2002,
   2018,
   2010,
   2012,
   1994,
   2014,
   2004,
   1990,
   2016,
   2000],
  'party': None,
  'state': 'WA',
  'tr

Pretty clear one of those is not Microsoft... so let's get the record for the one that is.

In [7]:
q_committee = {
    'committee_id': 'C00227546',
}
[r for r in all_results('/committee/C00227546/', q_committee)]

2018-07-17 22:34:16,697 : INFO : querying endpoint: http://api.open.fec.gov/v1/committee/C00227546/
2018-07-17 22:34:17,148 : INFO : 1 pages to be retrieved, with 1 records
2018-07-17 22:34:17,150 : INFO : all pages retrieved


[{'designation_full': 'Lobbyist/Registrant PAC',
  'custodian_city': 'REDMOND',
  'custodian_name_full': 'EATON, KELLY',
  'organization_type_full': 'Corporation',
  'committee_id': 'C00227546',
  'treasurer_phone': '2022635900',
  'state_full': 'Washington',
  'state': 'WA',
  'custodian_name_1': 'KELLY',
  'organization_type': 'C',
  'website': None,
  'party_full': None,
  'custodian_zip': '98052',
  'email': 'EDINGLE@MICROSOFT.COM;KELLY.EATON@MICROSOFT.COM',
  'party_type': None,
  'treasurer_name_middle': None,
  'treasurer_state': 'DC',
  'zip': '98052',
  'treasurer_name_2': 'INGLE',
  'treasurer_name_suffix': None,
  'treasurer_zip': '20001',
  'party_type_full': None,
  'treasurer_name_prefix': None,
  'treasurer_street_1': '901 K STREET, NW',
  'leadership_pac': None,
  'city': 'REDMOND',
  'committee_type_full': 'PAC - Qualified',
  'name': 'MICROSOFT CORPORATION POLITICAL ACTION COMMITTEE',
  'lobbyist_registrant_pac': 'E',
  'cycles': [2008,
   2006,
   1998,
   1988,
   1

Let's try getting some history data for 2018

In [8]:
count_results('/committee/C00227546/history/',{'cycle':2018})

16

In [9]:
[r for r in all_results('/committee/C00227546/history/',{'cycle':2018})]

2018-07-17 22:35:36,888 : INFO : querying endpoint: http://api.open.fec.gov/v1/committee/C00227546/history/
2018-07-17 22:35:37,284 : INFO : 1 pages to be retrieved, with 16 records
2018-07-17 22:35:37,286 : INFO : all pages retrieved


[{'committee_type_full': 'PAC - Qualified',
  'party_full': None,
  'candidate_ids': [],
  'filing_frequency': 'M',
  'cycle': 2018,
  'state': 'WA',
  'street_1': '3720 159TH AVENUE, NORTHEAST',
  'name': 'MICROSOFT CORPORATION POLITICAL ACTION COMMITTEE',
  'designation_full': 'Lobbyist/Registrant PAC',
  'cycles': [2008,
   2006,
   1998,
   1988,
   1996,
   1992,
   2002,
   2018,
   2010,
   2012,
   1994,
   2014,
   2004,
   1990,
   2016,
   2000],
  'organization_type': 'C',
  'designation': 'B',
  'committee_id': 'C00227546',
  'organization_type_full': 'Corporation',
  'zip': '98052',
  'party': None,
  'city': 'REDMOND',
  'committee_type': 'Q',
  'street_2': 'BUILDING 34, ROOM 4677',
  'treasurer_name': 'INGLE, EDWARD',
  'state_full': 'Washington'},
 {'committee_type_full': 'PAC - Qualified',
  'party_full': None,
  'candidate_ids': [],
  'filing_frequency': 'M',
  'cycle': 2016,
  'state': 'WA',
  'street_1': '16011 NE 36TH WAY',
  'name': 'MICROSOFT CORPORATION POLITIC

Not to interesting...

At this point I pivoted to start looking at data from within the exported csv, first to figure out how to query committee data. 

In [11]:
[r for r in all_results('/committee/C00008664/', {})]

2018-07-17 22:37:35,363 : INFO : querying endpoint: http://api.open.fec.gov/v1/committee/C00008664/
2018-07-17 22:37:36,271 : INFO : 1 pages to be retrieved, with 1 records
2018-07-17 22:37:36,273 : INFO : all pages retrieved


[{'treasurer_name_suffix': None,
  'fax': None,
  'treasurer_name_1': 'PATRICIA',
  'filing_frequency': 'M',
  'form_type': 'F1',
  'treasurer_name_2': 'THOMAS',
  'custodian_name_prefix': None,
  'last_file_date': '2018-07-04',
  'cycles': [2016,
   2006,
   1996,
   2014,
   1988,
   1984,
   2000,
   1982,
   1976,
   1980,
   2008,
   2012,
   1990,
   2018,
   2010,
   1978,
   2004,
   1994,
   1992,
   2002,
   1986,
   1998],
  'organization_type': None,
  'designation': 'U',
  'committee_id': 'C00008664',
  'custodian_name_2': None,
  'treasurer_state': 'MO',
  'treasurer_name_middle': None,
  'treasurer_name_prefix': None,
  'custodian_street_1': None,
  'committee_type': 'Y',
  'state': 'MO',
  'street_2': 'SUITE 11',
  'state_full': 'Missouri',
  'lobbyist_registrant_pac': None,
  'committee_type_full': 'Party - Qualified',
  'party_full': 'REPUBLICAN PARTY',
  'candidate_ids': [],
  'party_type_full': 'STATE',
  'treasurer_zip': '651091035',
  'first_file_date': '1975-03-1

Lot of stuff in there, but I just want a few fields.

In [12]:
committee = [r for r in all_results('/committee/C00403592/', {})]
committee_df = pd.DataFrame(committee)
committee_df.head()

2018-07-17 22:38:15,227 : INFO : querying endpoint: http://api.open.fec.gov/v1/committee/C00403592/
2018-07-17 22:38:15,626 : INFO : 1 pages to be retrieved, with 1 records
2018-07-17 22:38:15,628 : INFO : all pages retrieved


,candidate_ids,city,committee_id,committee_type,committee_type_full,custodian_city,custodian_name_1,custodian_name_2,custodian_name_full,custodian_name_middle,...,treasurer_name_prefix,treasurer_name_suffix,treasurer_name_title,treasurer_phone,treasurer_state,treasurer_street_1,treasurer_street_2,treasurer_zip,website,zip
0,[],CRANSTON,C00403592,Q,PAC - Qualified,WASHINGTON,JUDITH,ZAMORE,"ZAMORE, JUDITH",None,...,None,None,TREASURER,2025446960,DC,918 PENNSYLVANIA AVE SE,None,20003,None,02920


And to get just specific fields...

In [13]:
committee_df[['name','state', 'party_full', 'party_type_full', 'designation_full', 'committee_type_full', 'website']]

,name,state,party_full,party_type_full,designation_full,committee_type_full,website
0,NARRAGANSETT BAY PAC,RI,None,None,Leadership PAC,PAC - Qualified,None


The trouble with that is I really just want the value to put into a new CSV to merge with the original one.

There is a reason for this if statement. In working through the export to csv I hit a problem where some of the values come back with the text "None". Python interprets that as the None type. I don't really know Python so it took some finagling (and begging for help on Twitter) to get the syntax right on this check.

In [14]:
if committee_df.iloc[0]['party_full'] is not None:
    print(committee_df.iloc[0]['party_full'] )
else:
    print('yes')

yes


Now on to getting candidate data

In [15]:
candidate = [r for r in all_results('/candidate/H0CA27085/', {})]
candidate_df = pd.DataFrame(candidate)
candidate_df.head()

2018-07-17 22:41:55,708 : INFO : querying endpoint: http://api.open.fec.gov/v1/candidate/H0CA27085/
2018-07-17 22:41:56,266 : INFO : 1 pages to be retrieved, with 1 records
2018-07-17 22:41:56,267 : INFO : all pages retrieved


,active_through,address_city,address_state,address_street_1,address_street_2,address_zip,candidate_id,candidate_inactive,candidate_status,cycles,...,incumbent_challenge_full,last_f2_date,last_file_date,load_date,name,office,office_full,party,party_full,state
0,2018,BURBANK,CA,150 EAST OLIVE AVE,APT 208,91502,H0CA27085,False,C,"[2000, 2002, 2004, 2006, 2008, 2010, 2012, 201...",...,Incumbent,2017-05-15,2017-05-15,2018-04-11T21:09:46,"SCHIFF, ADAM",H,House,DEM,DEMOCRATIC PARTY,CA


In [16]:
candidate_df[['name', 'office', 'state', 'party_full']]

,name,office,state,party_full
0,"SCHIFF, ADAM",H,CA,DEMOCRATIC PARTY


In [18]:
party = candidate_df.iloc[0]['party_full']
print(party)

DEMOCRATIC PARTY


On to building a csv with the data I want. I first prepped a file of just unique committee and candidate ids using Excel.

The code is pretty straightforward, now that it works. I worked through this by figuring out how toloop over my existing csv and output to a new one first. Then I went back and worked the API calls in. There was a lot of fussing with this and I know it isn't optimal. I'm sure I'll learn better ways later.

In [19]:
import csv

In [ ]:
csv_file = open("mspac_with_party.csv", 'w', newline='')
writer = csv.writer(csv_file, escapechar=' ', quoting=csv.QUOTE_NONE)
writer.writerow(['recipient_committee_id', 'recipient_name', 'recipient_state', 'recipient_city', 'recipient_zip', \
                 'entity_type', 'entity_type_desc', 'election_type', 'fec_election_type_desc', 'fec_election_year', \
                 'committee_party_full', 'party_type_full', 'designation_full', 'committee_type_full', \
                 'candidate_party_full', 'candidate_id', 'candidate_name', 'candidate_office_description', 'candidate_office_district'])                
                   
current_line = []
with open('mspac-full.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        current_line = row['recipient_committee_id'] + ',' + row['recipient_name'].replace(',', '') + ',' + row['recipient_state'] + ',' \
                        + row['recipient_city'] + ',' + row['recipient_zip'] + ',' \
                        + row['entity_type'] + ',' + row['entity_type_desc'] + ',' \
                        + row['election_type'] + ',' + row['fec_election_type_desc'] + ',' + row['fec_election_year']
        # committee info
        endpoint = '/committee/{c}'.format(c=row['recipient_committee_id'])
        committee = all_results(endpoint, {})
        committee_df = pd.DataFrame(committee)
        if committee_df.iloc[0]['party_full'] is not None:
            current_line = current_line + ',' + committee_df.iloc[0]['party_full']
        else:
            current_line = current_line + ','
        if committee_df.iloc[0]['party_type_full'] is not None:
            current_line = current_line + ',' + committee_df.iloc[0]['party_type_full']
        else:
            current_line = current_line + ',' 
        if committee_df.iloc[0]['designation_full'] is not None:
            current_line = current_line + ',' + committee_df.iloc[0]['designation_full']
        else:
            current_line = current_line + ',' 
        if committee_df.iloc[0]['committee_type_full'] is not None:
            current_line = current_line + ',' + committee_df.iloc[0]['committee_type_full']
        else:
                current_line = current_line + ',' 
        #when there is a candidate get info
        if row['candidate_id'] is not '':
            c_endpoint = '/candidate/{c}'.format(c=row['candidate_id'])
            candidate = all_results(c_endpoint, {})
            candidate_df = pd.DataFrame(candidate)
            if candidate_df.iloc[0]['party_full'] is not None:
                current_line = current_line + ',' + candidate_df.iloc[0]['party_full']
            else:
                current_line = current_line + ',' 
            current_line = current_line + ',' + row['candidate_id'] + ',' + row['candidate_name'].replace(',', '') + ',' \
                            + row['candidate_office_description'] + ',' + row['candidate_office_district']
                            
        writer.writerow([current_line])

csv_file.close()
print('Done')
 

## Outcome
This got me most of the way there. I pulled this data back into Excel in a new table then created a data model to use pivot tables to analyze the spending data with political parties included. Some of the PACs did not have a party affiliation in the FEC data. I came to my own conclusions based on their spending records. I've made that clear with a column next to the committee party in the worksheet.

I accept that my analysis here is rough, but it seems fair based on what I'm seeing.

So far in the 2018 cycle the Microsoft PAC is giving more money and more contributions to Republicans. This is pretty shocking given how much legal assistance our company is providing to defend our own employees from policies put in place by the Republican party which controls all brnaches of goverment in the United States. So I don't know who this PAC is representing, but it does not seem to represent the interests of Microsoft or it's employees very well.

## Next steps
I'd like to go at this again more methodically, probably with bulk data exports. I'd love to get it to the point with any committee id as a starting input it could spit out the full analysis on its own. I think that would be extremely interesting to look at other corporste PACs to get some transparency on them. I suspect many of them, such as the Microsoft PAC, don't represent what the public face of the company is.

I'd also like to algorithmically determine the political orientation of a PAC by analyzing its spending. It would also be interesting to look at effectiveness of a PAC. Some of the unafiliated ones I looked up on OpenSecrets were shocking in how little they spent on campaigns, it was almost all fundraising and administartive overhead. It would be interesting to know if your corporate PAC was giving to such groups wouldn't it?

I'd also like to look at contributions to the PAC and cross check the political orientation of the contributors. That would be interesting to see if the PAC is at least representing its contributors even if it may be doing a poor job of representing its company affiliation. Which would be a good reason for these things to drop the corporate name they bear since that isn't who they really represent.